In [1]:
# imports
import os
import time
import datetime
import csv
import re

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from elasticsearch import Elasticsearch, helpers
from sklearn.feature_extraction.text import CountVectorizer
from joblib import Parallel, delayed

In [2]:
# Elastic host
es = Elasticsearch(
    hosts=[
            "https://localhost:9200"
    ],
    http_auth=("elastic", "NES9DZ-QwhanXAQf9caV"),
#     use_ssl=True,
    verify_certs=False,
#     ca_certs="./ca.crt"
    timeout=30, 
    max_retries=10, 
    retry_on_timeout=True
)

# dir
translated_csv_dir = r"D:\Debate_a_Base\Data\Parlamint_3.0\Csv_set"

In [3]:
# it is VERY dangerous to uncomment this
# es.indices.delete(index='ngrams')
# es.indices.delete(index='ngrams2')

In [4]:
# return list met dict [{Land1}, {Land 1}]
def get_csv_files(country_selection):
    os.chdir(translated_csv_dir)
    
    country_return_list = []

    # loop door alle folders die hierboven zijn geprint
    for country in os.listdir():
        
        # filter op specifiek land (IN BOX 2)
        if country in country_selection:
            paths_dict = {}

            # ga door alle inhoud van de landfolder heen
            for root, dirs, files in os.walk(country):
                file_data = []
                
                # loop door files van een folder
                for file in files:
                    
                    file_data.append(file)

                paths_dict[root] = file_data

            country_return_list.append(paths_dict)
        
    return country_return_list

In [5]:
# haal jaar maand en dag uit filename
def extract_file_date(file_name):
    
    year_month_day = re.search(r"\d{4}-\d{2}-\d{2}", file_name)
#     year, month, day = year_month_day[0].split("-")
    
    return year_month_day[0]

### Shingles uit CSV

In [6]:
# geeft een dict met dates als keys en files als vals
def get_files_per_date(country, after=None):
    files = get_csv_files(country)[0][country]
    
    dates_dict = {}
    
    if after is not None:
        
        after = time.mktime(datetime.datetime.strptime(after, "%Y-%m-%d").timetuple())
    
    # loop door alle files heen
    for file in files:
        
        date = extract_file_date(file)
        
        # filter al geuploade dates uit de dict
        if after is not None:
            timestamp = time.mktime(datetime.datetime.strptime(date, "%Y-%m-%d").timetuple())
            
            if (timestamp - after) <= 0:
                
                continue
        
        if date in dates_dict.keys():
            
            dates_dict[date] += [file]
            
        else:
            
            dates_dict[date] = [file]
            
    return dates_dict

In [7]:
# maakt paths aan voor alle files die bij een date horen
def get_date_paths(root, files):
    os.chdir(translated_csv_dir)
    
    paths = []
    
    for file in files:
        paths.append(os.path.join(root, file))
        
    return paths

In [8]:
# maakt ngrams van alle tekst in een csv file
def get_date_vocabulary(paths):
    os.chdir(translated_csv_dir)
    
    vocabulary = np.array(())
    
    for path in paths:
        df = pd.read_csv(path)
    
        # slecht vertaalde notebooks met alleen een index negeren
        if len(df.index) == 0:
            continue
            
        lines = [x for x in df["value"].to_list() if str(x) != 'nan']
    
        victor = CountVectorizer(ngram_range=(1, 5), token_pattern=r"(?u)\b\w+\b")
        victor.fit_transform(lines)
        
        vocabulary = np.unique(np.hstack((vocabulary, victor.get_feature_names_out()))) 
    
    return vocabulary

In [9]:
def get_date_counts(paths, vocabulary):
    os.chdir(translated_csv_dir)
    
    counts = np.zeros(shape=(len(vocabulary)), dtype=int)
    
    for path in paths:
        df = pd.read_csv(path)
    
        # slecht vertaalde notebooks met alleen een index negeren
        if len(df.index) == 0:
            continue
            
        lines = [x for x in df["value"].to_list() if str(x) != 'nan']
    
        victor = CountVectorizer(ngram_range=(1, 5), token_pattern=r"(?u)\b\w+\b")
        victor.fit_transform(vocabulary)
        
        counts = counts + np.sum(victor.transform(lines).toarray(), axis=0)
    
    return counts

In [10]:
# TODO: TIJDELIJKE OPLOSSING "index=ngrmams2" was eerst ="ngrams"
def get_es_date_dict(vocabulary, country, date, counts, index="ngrams2"):
    year, month, day = re.search(r"\d{4}-\d{2}-\d{2}", date)[0].split("-")
    
    bulk = []
    
    for ngram, count in zip(vocabulary, counts):
        content_dict, es_content_dict = {}, {}
        
        # entry inhoud
        content_dict["ngram"] = ngram
        content_dict["country"] = country
        content_dict["year"] = year
        content_dict["month"] = month
        content_dict["day"] = day
        content_dict["count"] = count
        
        # entry technische dingen
        es_content_dict["_index"] = index
        es_content_dict["_source"] = content_dict
        
        # voeg entry toe aan bulk return lijst
        bulk.append(es_content_dict)
        
    return bulk

In [11]:
# # functie testruimte
# t_folder = "BE"
# t_date = '2014-11-18'

# test_dates_dict = get_files_per_date(t_folder)
# print(test_dates_dict)
# test_paths = get_date_paths(t_folder, test_dates_dict[t_date ])
# print(test_paths)
# test_vocabulary = get_date_vocabulary(test_paths)
# print(test_vocabulary)
# test_counts = get_date_counts(test_paths, test_vocabulary)
# print(test_counts)
# test_date_es_dict = get_es_date_dict(test_vocabulary, t_folder, t_date, test_counts, 0)
# print(test_date_es_dict)

### Error management

In [12]:
log_loc = os.path.join(f"D:\Debate_a_Base\Data\Parlamint_3.0", "uploaded_ngram_dates.csv")

# houd bij welke files zijn geupload tijdens multiprocessing
def log_date_upload(country, date):
    with open(log_loc, 'a', newline='', encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow([country, date])

In [13]:
# filter de dates die al indexed zijn
def get_remaining_dates(country, dates):
    df_processed_dates = pd.read_csv(os.path.join(f"D:\Debate_a_Base\Data\Parlamint_3.0", "uploaded_ngram_dates.csv"))
    
    df_processed_dates_cutout = df_processed_dates[df_processed_dates["key"] == country]
    
    if len(df_processed_dates_cutout) != 0:
        disposable_dates = list(df_processed_dates_cutout["value"])
        
        for disposable_date in disposable_dates:
            if disposable_date in dates.keys():
                dates.pop(disposable_date)
                
    return dates

### Countvectorizer 1.3
Elastic (index = ngrams):
- | Shingle | Land | Jaar | Maand | Dag | Aantal | Percentage |
 - Shingles doen voor een datum, voor een land
 - Percentage berekenen
 - Uploaden in ES

In [14]:
def upload_ngrams(countries):
    
    for country in countries:
        
        # vraag de dates op die geprocessed moeten worden
        dates = get_remaining_dates(country, get_files_per_date(country))
        if len(dates) == 0:
            continue
            
        # dingen voor overzicht in prints
        timer = time.time()
        total_dates = len(dates.keys())
        
        def multithread_dates(date):        
            paths = get_date_paths(country, dates[date])
            
            vocabulary = get_date_vocabulary(paths)
            counts = get_date_counts(paths, vocabulary)
            
            elastic_dict = get_es_date_dict(vocabulary, country, date, counts)
            
            helpers.bulk(es, elastic_dict)
            log_date_upload(country, date)
            print(f"[Info]: {round((time.time() - timer) / 60, 2)}m, Uploaded: {date} {country}")
            
        # process multiple dates at the same time
        Parallel(n_jobs=15, prefer="threads")(delayed(multithread_dates)(date) for date in dates)
            
        print(f"[Info]: {country} finished")
        
    return True

In [15]:
# werkende landen
todo = ['AT', 'BA', 'BE', 'BG', 'CZ', 'DK', 'EE', 'ES-CT', 'ES-GA', 'FR', 'GR', 'HR', 'HU', 'IS', 'IT', 'LV', 'NL', 'NO', 'PL', 'PT', 'RS', 'SE', 'SI', 'TR', 'UA', 'GB']

done = []

# GEBRUIKT 2 INDEXES: ngrams, ngrams2
# is nog niet ingeprogrammeerd, maar tot nu toe handmatig

upload = upload_ngrams(todo)

print(upload)

[Info]: 0.42m, Uploaded: 2003-01-14 NO
[Info]: 0.47m, Uploaded: 2002-01-17 NO
[Info]: 0.54m, Uploaded: 2009-01-13 NO
[Info]: 0.55m, Uploaded: 2012-01-11 NO
[Info]: 0.55m, Uploaded: 2007-01-17 NO
[Info]: 0.67m, Uploaded: 2016-01-13 NO
[Info]: 0.69m, Uploaded: 2015-01-13 NO
[Info]: 0.76m, Uploaded: 2000-01-20 NO
[Info]: 0.78m, Uploaded: 2013-01-16 NO
[Info]: 0.84m, Uploaded: 2007-01-18 NO
[Info]: 0.84m, Uploaded: 2002-01-22 NO
[Info]: 0.9m, Uploaded: 2018-01-17 NO
[Info]: 0.93m, Uploaded: 2010-01-13 NO
[Info]: 0.96m, Uploaded: 2002-01-16 NO
[Info]: 0.97m, Uploaded: 2019-01-16 NO
[Info]: 1.11m, Uploaded: 2001-01-23 NO
[Info]: 1.17m, Uploaded: 2000-01-19 NO
[Info]: 1.17m, Uploaded: 2017-01-17 NO
[Info]: 1.24m, Uploaded: 2009-01-09 NO
[Info]: 1.28m, Uploaded: 2011-01-13 NO
[Info]: 1.31m, Uploaded: 2015-01-14 NO
[Info]: 1.42m, Uploaded: 2010-01-14 NO
[Info]: 1.46m, Uploaded: 2014-01-15 NO
[Info]: 1.48m, Uploaded: 2008-01-15 NO
[Info]: 1.67m, Uploaded: 2006-01-31 NO
[Info]: 1.73m, Uploaded: 2

[Info]: 13.02m, Uploaded: 2016-02-16 NO
[Info]: 13.1m, Uploaded: 1998-11-27 NO
[Info]: 13.22m, Uploaded: 2020-02-11 NO
[Info]: 13.23m, Uploaded: 2008-02-06 NO
[Info]: 13.24m, Uploaded: 2002-02-14 NO
[Info]: 13.33m, Uploaded: 2015-02-10 NO
[Info]: 13.51m, Uploaded: 2004-02-10 NO
[Info]: 13.59m, Uploaded: 2005-02-18 NO
[Info]: 13.83m, Uploaded: 2018-02-07 NO
[Info]: 13.84m, Uploaded: 2015-02-05 NO
[Info]: 13.86m, Uploaded: 2002-02-13 NO
[Info]: 13.95m, Uploaded: 1999-02-03 NO
[Info]: 14.08m, Uploaded: 2017-02-14 NO
[Info]: 14.17m, Uploaded: 2003-02-06 NO
[Info]: 14.22m, Uploaded: 2009-02-12 NO
[Info]: 14.29m, Uploaded: 2008-02-07 NO
[Info]: 14.3m, Uploaded: 2016-02-18 NO
[Info]: 14.35m, Uploaded: 2016-02-17 NO
[Info]: 14.49m, Uploaded: 2019-01-30 NO
[Info]: 14.54m, Uploaded: 2020-02-12 NO
[Info]: 14.7m, Uploaded: 2006-02-28 NO
[Info]: 14.71m, Uploaded: 2011-02-08 NO
[Info]: 14.73m, Uploaded: 2002-02-19 NO
[Info]: 14.76m, Uploaded: 2010-02-11 NO
[Info]: 14.94m, Uploaded: 2021-02-04 NO
[In

[Info]: 29.68m, Uploaded: 2006-03-16 NO
[Info]: 29.68m, Uploaded: 2012-03-01 NO
[Info]: 29.83m, Uploaded: 2004-03-05 NO
[Info]: 29.87m, Uploaded: 2013-03-13 NO
[Info]: 30.21m, Uploaded: 2011-02-23 NO
[Info]: 30.24m, Uploaded: 2015-03-12 NO
[Info]: 30.29m, Uploaded: 2002-03-06 NO
[Info]: 30.55m, Uploaded: 2020-03-05 NO
[Info]: 30.59m, Uploaded: 2012-03-07 NO
[Info]: 30.67m, Uploaded: 2007-03-19 NO
[Info]: 30.74m, Uploaded: 2019-02-19 NO
[Info]: 30.97m, Uploaded: 2005-03-17 NO
[Info]: 30.97m, Uploaded: 2009-03-16 NO
[Info]: 31.19m, Uploaded: 2002-03-07 NO
[Info]: 31.24m, Uploaded: 2008-03-03 NO
[Info]: 31.33m, Uploaded: 2014-03-20 NO
[Info]: 31.35m, Uploaded: 2019-02-20 NO
[Info]: 31.39m, Uploaded: 2013-03-14 NO
[Info]: 31.5m, Uploaded: 2017-03-08 NO
[Info]: 31.51m, Uploaded: 2020-03-10 NO
[Info]: 31.73m, Uploaded: 2004-03-09 NO
[Info]: 31.75m, Uploaded: 2010-03-04 NO
[Info]: 31.93m, Uploaded: 2007-03-20 NO
[Info]: 31.96m, Uploaded: 2012-03-08 NO
[Info]: 32.1m, Uploaded: 2003-02-25 NO
[I

[Info]: 47.32m, Uploaded: 2006-04-27 NO
[Info]: 47.46m, Uploaded: 2007-04-16 NO
[Info]: 47.53m, Uploaded: 2018-03-20 NO
[Info]: 47.57m, Uploaded: 2017-03-30 NO
[Info]: 47.67m, Uploaded: 2015-03-24 NO[Info]: 47.67m, Uploaded: 2020-04-22 NO

[Info]: 47.76m, Uploaded: 2005-04-19 NO
[Info]: 47.82m, Uploaded: 2008-03-14 NO
[Info]: 47.88m, Uploaded: 1999-03-18 NO
[Info]: 48.1m, Uploaded: 1999-03-17 NO
[Info]: 48.11m, Uploaded: 2019-03-14 NO
[Info]: 48.26m, Uploaded: 2008-04-01 NO
[Info]: 48.39m, Uploaded: 2018-03-21 NO
[Info]: 48.41m, Uploaded: 2022-03-18 NO
[Info]: 48.5m, Uploaded: 2013-03-22 NO
[Info]: 48.58m, Uploaded: 2006-05-02 NO
[Info]: 48.73m, Uploaded: 2000-03-23 NO
[Info]: 48.74m, Uploaded: 2014-04-10 NO
[Info]: 48.82m, Uploaded: 2003-03-26 NO
[Info]: 48.87m, Uploaded: 2013-03-21 NO
[Info]: 48.87m, Uploaded: 2002-04-10 NO
[Info]: 48.89m, Uploaded: 2014-04-23 NO
[Info]: 49.08m, Uploaded: 2001-03-28 NO
[Info]: 49.13m, Uploaded: 2017-04-03 NO
[Info]: 49.32m, Uploaded: 2012-04-11 NO
[I

[Info]: 64.94m, Uploaded: 2002-04-29 NO
[Info]: 65.01m, Uploaded: 2019-04-09 NO
[Info]: 65.1m, Uploaded: 2020-05-07 NO
[Info]: 65.11m, Uploaded: 2006-05-22 NO
[Info]: 65.35m, Uploaded: 2001-04-25 NO
[Info]: 65.38m, Uploaded: 2003-04-11 NO
[Info]: 65.42m, Uploaded: 2016-05-03 NO
[Info]: 65.57m, Uploaded: 2010-04-14 NO
[Info]: 65.57m, Uploaded: 2004-05-12 NO
[Info]: 65.69m, Uploaded: 2012-05-03 NO
[Info]: 65.95m, Uploaded: 2021-03-23 NO
[Info]: 66.06m, Uploaded: 2021-03-24 NO
[Info]: 66.23m, Uploaded: 2016-05-04 NO
[Info]: 66.42m, Uploaded: 2002-04-30 NO
[Info]: 66.43m, Uploaded: 2009-05-06 NO
[Info]: 66.46m, Uploaded: 2018-04-12 NO
[Info]: 66.58m, Uploaded: 2005-05-27 NO
[Info]: 66.69m, Uploaded: 2000-05-02 NO
[Info]: 66.78m, Uploaded: 2003-04-29 NO
[Info]: 66.89m, Uploaded: 2017-04-26 NO
[Info]: 66.9m, Uploaded: 2009-05-07 NO
[Info]: 66.96m, Uploaded: 2004-05-13 NO
[Info]: 67.03m, Uploaded: 2015-05-05 NO
[Info]: 67.17m, Uploaded: 2018-04-18 NO
[Info]: 67.2m, Uploaded: 2022-04-07 NO
[In

...

[Info]: 76.55m, Uploaded: 2016-10-19 GB
[Info]: 76.69m, Uploaded: 2016-10-12 GB
[Info]: 76.98m, Uploaded: 2016-10-18 GB
[Info]: 77.24m, Uploaded: 2016-10-13 GB
[Info]: 77.33m, Uploaded: 2016-10-28 GB
[Info]: 77.61m, Uploaded: 2016-10-24 GB
[Info]: 77.72m, Uploaded: 2016-10-20 GB
[Info]: 78.37m, Uploaded: 2016-11-04 GB
[Info]: 78.95m, Uploaded: 2016-11-14 GB
[Info]: 79.13m, Uploaded: 2016-11-09 GB
[Info]: 79.29m, Uploaded: 2016-10-25 GB
[Info]: 79.51m, Uploaded: 2016-11-01 GB
[Info]: 79.91m, Uploaded: 2016-10-26 GB
[Info]: 80.23m, Uploaded: 2016-10-27 GB
[Info]: 80.49m, Uploaded: 2016-10-31 GB
[Info]: 80.7m, Uploaded: 2016-11-02 GB
[Info]: 81.08m, Uploaded: 2016-11-08 GB
[Info]: 81.17m, Uploaded: 2016-11-18 GB
[Info]: 81.55m, Uploaded: 2016-11-15 GB
[Info]: 81.68m, Uploaded: 2016-11-25 GB
[Info]: 81.71m, Uploaded: 2016-11-07 GB
[Info]: 82.31m, Uploaded: 2016-11-03 GB
[Info]: 82.53m, Uploaded: 2016-11-16 GB
[Info]: 82.95m, Uploaded: 2016-11-17 GB
[Info]: 83.18m, Uploaded: 2016-12-09 GB
[

[Info]: 136.1m, Uploaded: 2018-02-07 GB
[Info]: 136.5m, Uploaded: 2018-02-23 GB
[Info]: 136.55m, Uploaded: 2018-02-06 GB
[Info]: 136.6m, Uploaded: 2018-01-30 GB
[Info]: 136.61m, Uploaded: 2018-02-08 GB
[Info]: 137.34m, Uploaded: 2018-01-31 GB
[Info]: 137.42m, Uploaded: 2018-02-05 GB
[Info]: 138.17m, Uploaded: 2018-02-20 GB
[Info]: 138.2m, Uploaded: 2018-02-22 GB
[Info]: 138.36m, Uploaded: 2018-02-21 GB
[Info]: 139.01m, Uploaded: 2018-02-27 GB
[Info]: 139.4m, Uploaded: 2018-03-01 GB
[Info]: 139.57m, Uploaded: 2018-02-26 GB
[Info]: 139.83m, Uploaded: 2018-03-16 GB
[Info]: 140.06m, Uploaded: 2018-02-28 GB
[Info]: 140.52m, Uploaded: 2018-03-06 GB
[Info]: 140.75m, Uploaded: 2018-03-23 GB
[Info]: 140.88m, Uploaded: 2018-03-13 GB
[Info]: 141.32m, Uploaded: 2018-03-05 GB
[Info]: 141.68m, Uploaded: 2018-03-08 GB
[Info]: 141.99m, Uploaded: 2018-03-20 GB
[Info]: 142.47m, Uploaded: 2018-03-12 GB
[Info]: 142.64m, Uploaded: 2018-03-15 GB
[Info]: 142.7m, Uploaded: 2018-03-07 GB
[Info]: 143.5m, Upload

[Info]: 194.45m, Uploaded: 2019-04-03 GB
[Info]: 194.66m, Uploaded: 2019-04-10 GB
[Info]: 194.72m, Uploaded: 2019-03-28 GB
[Info]: 194.73m, Uploaded: 2019-04-08 GB
[Info]: 194.94m, Uploaded: 2019-04-09 GB
[Info]: 195.37m, Uploaded: 2019-04-04 GB
[Info]: 195.66m, Uploaded: 2019-04-11 GB
[Info]: 196.42m, Uploaded: 2019-04-29 GB
[Info]: 196.54m, Uploaded: 2019-04-24 GB
[Info]: 196.83m, Uploaded: 2019-04-30 GB
[Info]: 197.11m, Uploaded: 2019-04-25 GB
[Info]: 197.55m, Uploaded: 2019-05-13 GB
[Info]: 197.66m, Uploaded: 2019-05-01 GB
[Info]: 197.98m, Uploaded: 2019-05-02 GB
[Info]: 198.2m, Uploaded: 2019-05-07 GB
[Info]: 198.38m, Uploaded: 2019-05-08 GB
[Info]: 198.97m, Uploaded: 2019-05-14 GB
[Info]: 199.01m, Uploaded: 2019-05-20 GB
[Info]: 199.44m, Uploaded: 2019-06-14 GB
[Info]: 199.54m, Uploaded: 2019-05-15 GB
[Info]: 199.82m, Uploaded: 2019-05-21 GB
[Info]: 199.92m, Uploaded: 2019-05-09 GB
[Info]: 200.23m, Uploaded: 2019-05-16 GB
[Info]: 200.54m, Uploaded: 2019-05-22 GB
[Info]: 200.71m, 

[Info]: 249.79m, Uploaded: 2020-09-21 GB
[Info]: 250.06m, Uploaded: 2020-09-24 GB
[Info]: 250.24m, Uploaded: 2020-10-09 GB
[Info]: 250.45m, Uploaded: 2020-09-17 GB
[Info]: 250.57m, Uploaded: 2020-09-22 GB
[Info]: 250.96m, Uploaded: 2020-09-23 GB
[Info]: 251.14m, Uploaded: 2020-09-29 GB
[Info]: 251.65m, Uploaded: 2020-10-01 GB
[Info]: 251.83m, Uploaded: 2020-09-28 GB
[Info]: 252.25m, Uploaded: 2020-09-30 GB
[Info]: 252.3m, Uploaded: 2020-10-16 GB
[Info]: 252.59m, Uploaded: 2020-10-05 GB
[Info]: 252.87m, Uploaded: 2020-10-07 GB
[Info]: 253.0m, Uploaded: 2020-10-12 GB
[Info]: 253.04m, Uploaded: 2020-10-23 GB
[Info]: 253.51m, Uploaded: 2020-10-06 GB
[Info]: 253.65m, Uploaded: 2020-10-08 GB
[Info]: 253.9m, Uploaded: 2020-10-15 GB
[Info]: 253.9m, Uploaded: 2020-10-27 GB
[Info]: 254.1m, Uploaded: 2020-10-26 GB
[Info]: 254.87m, Uploaded: 2020-10-29 GB
[Info]: 255.03m, Uploaded: 2020-10-21 GB
[Info]: 255.15m, Uploaded: 2020-10-28 GB
[Info]: 255.23m, Uploaded: 2020-10-14 GB
[Info]: 255.73m, Uplo

[Info]: 308.27m, Uploaded: 2021-12-16 GB
[Info]: 308.41m, Uploaded: 2022-01-10 GB
[Info]: 308.5m, Uploaded: 2022-01-05 GB
[Info]: 309.13m, Uploaded: 2022-01-21 GB
[Info]: 309.19m, Uploaded: 2022-01-06 GB
[Info]: 309.6m, Uploaded: 2022-01-11 GB
[Info]: 310.53m, Uploaded: 2022-01-13 GB
[Info]: 311.02m, Uploaded: 2022-01-17 GB
[Info]: 311.34m, Uploaded: 2022-01-18 GB
[Info]: 311.41m, Uploaded: 2022-01-19 GB
[Info]: 311.56m, Uploaded: 2022-01-28 GB
[Info]: 312.21m, Uploaded: 2022-01-24 GB
[Info]: 312.25m, Uploaded: 2022-01-20 GB
[Info]: 312.45m, Uploaded: 2022-01-25 GB
[Info]: 312.64m, Uploaded: 2022-01-27 GB[Info]: 312.64m, Uploaded: 2022-01-26 GB

[Info]: 313.66m, Uploaded: 2022-01-31 GB
[Info]: 313.78m, Uploaded: 2022-02-04 GB
[Info]: 314.24m, Uploaded: 2022-02-01 GB
[Info]: 314.27m, Uploaded: 2022-02-02 GB
[Info]: 314.96m, Uploaded: 2022-03-04 GB
[Info]: 315.0m, Uploaded: 2022-02-07 GB
[Info]: 315.12m, Uploaded: 2022-02-03 GB
[Info]: 316.1m, Uploaded: 2022-02-25 GB
[Info]: 316.44m, Upl